# Presentación

En el presente cuaderno se desarrollará la propuesta de un modulo en Python para la recomendacion de productos a un usuario nuevo, esto se hará basado en las compras realizadas por otros usuarios con caracteristicas similares bajo la metodologia de filtro colaborativo, permitiendo así obtener una mejor experiencia de compra basado en los intereses de estos.

# Importacion de librerias

Primeramente, importaremos las librerias que vamos a utilizar a lo largo del desarrollo de nuestro modulo, estas librerias son: 

1. numpy (NumPy): NumPy para el manejo eficiente de matrices y operaciones matemáticas.
2. seaborn (Seaborn): biblioteca para la visualización de datos con gráficos estéticos y personalizados.
3. pandas (Pandas): biblioteca Pandas para el análisis y manipulación de datos en estructuras de DataFrame y Series.
4. cosine_similarity (Scikit-learn): Función cosine_similarity de Scikit-learn para calcular la similitud coseno entre dos vectores.
5. matplotlib.pyplot (Matplotlib): Biblioteca Matplotlib para la generación de gráficos y visualizaciones de datos.
6. recometrics: Marco de evaluación independiente de la biblioteca para sistemas de recomendación de retroalimentación implícita que se basan en modelos de factorización matricial de bajo rango.
7. csr_matrix (scipy, sparse): Función para crear una matriz CSR (Fila dispersa comprimida. Para un corte rápido de filas, productos vectoriales de matriz más rápidos) pasando una matriz a la función
8. implicit: Proporciona implementaciones de varios algoritmos diferentes para sistemas de recomendación de retroalimentación implícita ademas de proporcionar metricas de evaluacion para los modelos.

In [4]:
import numpy as np
import seaborn as sns
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from implicit.evaluation import mean_average_precision_at_k, precision_at_k
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
import implicit
import recometrics
import os; os.environ['OPENBLAS_NUM_THREADS']='1'
from scipy.sparse import coo_matrix
import warnings
warnings.filterwarnings('ignore')

# Importación de los datos

En la seleccion de datos, se eligen las fuentes de datos. En el caso de iBuyFlowers, vamos a basarnos en informacion real de las tablas con año base 2023.
Estas son: 

* **purchase**: Contiene el historial de compras de un usuario (Customer).
* **state**: Guarda la informacion de cada estado como su nombre o ISO, Ej: New York - ISO -> (NY).
* **cart_box**: Contiene toda la informacion relacionada a la caja que incluye los elementos que un usuario compro, el total, la cantidad, el agricultor, la compra a la que esta relacionada, etc.
* **cart_box_item**: Contiene toda la informacion de un producto comprado, ej: La variedad, grupo al que pertenece un producto, tu precio, cantidad, la caja a la cual pertenece, etc.

Estas tablas son seleccionadas para poder realizar la recomendaciones basadas en compras por usuarios, debido contienen información sobre transacciones, cajas compradas e items.


A continuación, vamos a utilizar un dataset que contiene toda la informacion combinada de estas 4 tablas de todo el año 2023. Se leerá el dataset con la funcion `read_csv` de pandas y extraeremos toda la información posible.

In [5]:
df = pd.read_csv('./all_2023_ibf_data.csv', low_memory=False)
df.shape

(155807, 157)

Al realizar la lectura del archivo pudimos obtener 155807 Filas y 157 Columnas, que son el resultado de la combinacion de todas estas tablas.

# Preprocesamiento

En la etapa de pre procesamiento se llevan a cabo la limpieza y preparacion de los datos para su análisis. Esto incluye la eliminación de valores nulos, tratamiento de datos faltantes, conversión de formatos, y etc. En este contexto, es importante seleccionar las columnas relevantes: 

Por supuesto, aquí tienes la descripción de cada columna:

- `cart_box_item_cart_box_purchase_state_state`: Estado en el que reside un usuario asociado a una compra para su dirección de entrega.
- `cart_box_item_variety_variety_name`: Nombre de la variedad de un item en la caja.
- `cart_box_item_variety__KEY`: Identificador de la variedad de un item en la caja.
- `cart_box_item_length`: Longitud de un item, ej: 60 -> 60Cm.
- `cart_box_item_product_group_common_name`: Nombre común del grupo de productos al que pertenece el item.
- `cart_box_item_cart_box_customer_margin`: Margen del cliente en el negocio.
- `cart_box_item_cart_box_customer_tier_sbx`: categorización del cliente en el negocio.
- `cart_box_item_cart_box_customer__KEY`: Identificador del cliente del en la base de datos.
- `cart_box_item_cart_box_customer_business`: Tipo de negocio del cliente.
- `cart_box_item_cart_box_customer_events_per_year`: Eventos por año del cliente.
- `cart_box_item_cart_box_customer_stores_quantity`: Cantidad de tiendas del cliente.
- `cart_box_item_cart_box_customer_employees_quantity`: Cantidad de empleados del cliente.
- `cart_box_item_cart_box_customer_spend_per_week`: Gasto por semana del cliente.

Teniendo en cuenta los objetivo y las reglas del negocio, estas columnas estan relacionadas directamente con el resultado esperado, debido que es informacion que todo usuario ingresa al momento de registrarse en la plataforma de iBuyFlowers y esta predefinida en el sistema, por lo tanto siempre escogera una opcion dada por el negocio, lo cual permitira tener un mayor control de la informacion para la transformacion de datos y posterior aplicacion de la metodologia seleccionada para el sistema de recomendación.

In [6]:
# Columnas a trabajar en nuestro dataset
columns_selected = ['cart_box_item_cart_box_purchase_state_state', 'cart_box_item_variety_variety_name', 'cart_box_item_variety__KEY', 'cart_box_item_length',
                    'cart_box_item_product_group_common_name', 'cart_box_item_cart_box_customer_margin', 'cart_box_item_cart_box_customer_tier_sbx'
                    , 'cart_box_item_cart_box_customer__KEY', 'cart_box_item_cart_box_customer_business'
                    , 'cart_box_item_cart_box_customer_events_per_year', 'cart_box_item_cart_box_customer_stores_quantity', 'cart_box_item_cart_box_customer_employees_quantity',
                   'cart_box_item_cart_box_customer_spend_per_week']
df_selected = df[columns_selected]
df_selected.head()


,cart_box_item_cart_box_purchase_state_state,cart_box_item_variety_variety_name,cart_box_item_variety__KEY,cart_box_item_length,cart_box_item_product_group_common_name,cart_box_item_cart_box_customer_margin,cart_box_item_cart_box_customer_tier_sbx,cart_box_item_cart_box_customer__KEY,cart_box_item_cart_box_customer_business,cart_box_item_cart_box_customer_events_per_year,cart_box_item_cart_box_customer_stores_quantity,cart_box_item_cart_box_customer_employees_quantity,cart_box_item_cart_box_customer_spend_per_week
0,Pennsylvania,Dark X-Pression,469a7ea9-d706-4e90-adc4-f303e010ce0c,40.0,Garden Roses,36,tier_2,a6f63b8c-4ac6-493d-a1b8-0b0ed65d2b6f,Wedding & event floral designer only (not a fl...,25-59,0.0,2,$500-$999
1,Missouri,Pink Pigeon,4da78552-4952-4971-8ea5-f9ce645398e9,50.0,Mini Carnation (Spray),36,tier_2,b80950aa-01fa-493a-bfb3-4d5ec7150bef,Wedding & event floral designer only (not a fl...,> 60,1.0,2,NaN
2,Texas,Variegated,36b75d67-defd-4b5e-a3df-218a67d52d21,40.0,Aspidistra,36,tier_1,6636b4ad-5643-4283-8d23-073f1d6363ae,Florist shop,> 60,1.0,6,> $1000
3,New York,Deep Purple,7700981c-8639-4a6b-ac12-8f4577aac4ff,60.0,Roses,36,tier_2,d5d7c103-f029-4f25-ace1-56f15e76eb8d,Florist shop,25-59,2.0,4,NaN
4,Minnesota,Green,7477eac8-f207-41d6-b543-70c9845890b7,40.0,Hebes,36,tier_2,392b3117-0027-487a-8d11-a190c57b14e6,Florist shop,10-24,1.0,3,NaN


# Transformación

En esta etapa realizaremos una transformación en nuestros datos para conocerlos un poco mejor e ir haciendo el dataset mas legible, sin información faltante y valores repetidos que permitan un mejor analisis. 

En primera instancia vamos a renombrar las columnas, debido que tienen un nombre extenso que evita una mejor lectura a primera vista, dificulta la refactorizacion del codigo a futuro y pueden ser inconsistentes. 

In [7]:
copy_selected = df_selected.copy()

copy_selected.rename(columns={
    'cart_box_item_cart_box_purchase_state_state': 'state',
    'cart_box_item_cart_box_customer_margin': 'margin',
    'cart_box_item_cart_box_customer_tier_sbx': 'tier',
    'cart_box_item_cart_box_customer__KEY': "customer_key",
    'cart_box_item_variety_variety_name': 'variety',
        'cart_box_item_variety__KEY': 'variety_key',
    'cart_box_item_length': 'length',
    'cart_box_item_product_group_common_name': 'product_group',
    'cart_box_item_cart_box_customer_business': 'business',
    'cart_box_item_cart_box_customer_events_per_year':'events_per_year',
    'cart_box_item_cart_box_customer_stores_quantity':'stores_quantity',
    'cart_box_item_cart_box_customer_employees_quantity': 'employees_quantity',
    'cart_box_item_cart_box_customer_spend_per_week': 'spend_per_week'
    }, inplace=True)

# copy_selected.head()

Luego de transfomar nuestras columnas, vamos analizar los valores faltantes dentro de nuestro dataset para luego realizar una imputacion y asi reemplazarlos mediante un tratamiento de datos faltantes.

In [8]:
for col in copy_selected.columns:
    pct_missing = np.mean(copy_selected[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

state - 0%
variety - 0%
variety_key - 0%
length - 0%
product_group - 0%
margin - 0%
tier - 0%
customer_key - 0%
business - 0%
events_per_year - 0%
stores_quantity - 2%
employees_quantity - 0%
spend_per_week - 67%


Teniendo en cuenta lo anterior, se observa las columnas stores_quantity y spend_per_week con el 2% y el 67% de los datos faltantes respectivamente. Por lo tanto vamos a tratar los datos faltantes teniendo en cuenta las reglas del negocio y las columnas seleccionadas, esto se hará colocando como 'Unknown' todo valor faltante de las columnas de tipo categorico y 0.0 para todo valor faltante de las columnas de tipo numerico.

In [9]:
copy_selected.loc[copy_selected['spend_per_week'] == "",['spend_per_week']] =  'Unknown'
copy_selected.loc[copy_selected['spend_per_week'] != copy_selected['spend_per_week'],['spend_per_week']] =  'Unknown'
copy_selected.loc[copy_selected['events_per_year'] == "",['events_per_year']] =  'Unknown'
copy_selected.loc[copy_selected['stores_quantity'] == "",['stores_quantity']] =  0.0
copy_selected.loc[copy_selected['employees_quantity'] == "",['employees_quantity']] =  0.0
copy_selected.loc[:,['stores_quantity']] = copy_selected['stores_quantity'].fillna(0).astype(float)

Luego de hacer la imputación de valores, nuevamente realizamos una verificación en nuestro dataset para corroborar que no hay datos faltantes: 

In [10]:
for col in copy_selected.columns:
    pct_missing = np.mean(copy_selected[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

state - 0%
variety - 0%
variety_key - 0%
length - 0%
product_group - 0%
margin - 0%
tier - 0%
customer_key - 0%
business - 0%
events_per_year - 0%
stores_quantity - 0%
employees_quantity - 0%
spend_per_week - 0%


# Modelado

Luego de procesar, analizar e imputar valores faltantes en nuestro dataset, para realizar el modelo, nos vamos a basar en el filtrado colaborativo basado en "Retroalimentacion implicita" o "Implicit feedback", en el cual se tiene en cuenta el comportamiento en el pasado de un conjunto de usuarios, como por ejemplo el historico de compras, navegacion por las paginas del software, clicks realizados, etc. Esto es debido que no se cuenta con "Retroalimentacion explicita" o "Explicit feedback" por parte de los usuarios, como por ejemplo calificacion de un producto, comentarios, etc. 

Para este modelado se van a identificar caracteristicas de la retroalimentacion implicita, tomadas del comportamiento del usuario basado en su historial de compras, en este caso evaluaremos la confianza en un producto basado en la frecuencia de las acciones del usuario, como por ejemplo, numero de veces que ha comprado un producto.

Primeramente, vamos a tomar el conjunto de datos inicial, borraremos las columnas que no son necesarias y generaremos dos columnas nuevas, que permitan identificar al usuario, el producto y la frecuencia de compras de ese usuario con ese producto.

In [11]:
products_by_freq = copy_selected.copy()

products_by_freq['length'].fillna(0, inplace=True)
products_by_freq['product_name'] = products_by_freq.apply(lambda x: str(x.product_group) + " " + str(x.variety) + " " + str(int(float(x.length)))+ "cm", axis = 1)
products_by_freq = products_by_freq.drop(['variety_key', 'variety','length',
                                          'product_group', 'margin', 'tier',
                                          'state', 'business',
                                          'events_per_year', 
                                          'stores_quantity',
                                          'employees_quantity','spend_per_week'], axis='columns')

columns = ['product_name']
aggs = {key: ['count'] for key in columns}
products_by_freq = products_by_freq.groupby(by=["customer_key", 'product_name']).agg(aggs).droplevel(0,axis=1).reset_index().sort_values(by=["count"],ascending=False)

products_by_freq.head(5)

,customer_key,product_name,count
27811,45193aab-887a-4590-be6e-6558691e63b1,Mini Carnation (Spray) Hamada 50cm,196
27812,45193aab-887a-4590-be6e-6558691e63b1,Mini Carnation (Spray) Imagine 50cm,196
27813,45193aab-887a-4590-be6e-6558691e63b1,Mini Carnation (Spray) Pigeon 50cm,196
27814,45193aab-887a-4590-be6e-6558691e63b1,Mini Carnation (Spray) Pink Pigeon 50cm,196
27815,45193aab-887a-4590-be6e-6558691e63b1,Mini Carnation (Spray) Rony 50cm,196



Luego de obtener la frecuencia de compra de los productos por usuario, se aplicara la tecnica conocida como "mínimos cuadrados alternos" o "Alternating Least Squares algorithm", esta tecnica esta basada en factorizacion matricial, lo cual permite descomponer una matriz que contiene la relacion de usuario-articulo en dos matrices que tienen dimensiones inferiores, siendo una de ellas para usuarios y otra para los elementos. En esta implementacion se obtendra una nueva matriz que contiene los ID de los productos y clientes cuya multiplicacion da como un resultado aproximado el puntaje de la interaccion entre ellos

Primeramente, para el proceso de creacion de la matriz que tendra la relacion entre cliente y producto se crearon dos columnas de tipo numerico llamadas, "customer_id" e "item_id", en las cuales se asignara en valor numerico un ID para los productos y los clientes. Esto con el fin de crear nuestra matriz de factorizacion.

Procedemos con la creacion de las columnas:

In [12]:
unique_customers = products_by_freq['customer_key'].unique()
unique_items = products_by_freq['product_name'].unique()

customer_ids = {customer: idx for idx, customer in enumerate(unique_customers)}
item_ids = {item: idx for idx, item in enumerate(unique_items)}

products_by_freq['customer_id'] = products_by_freq['customer_key'].map(customer_ids)
products_by_freq['item_id'] = products_by_freq['product_name'].map(item_ids)

n_users = products_by_freq.customer_key.unique().shape[0]
n_items = products_by_freq.product_name.unique().shape[0]

print('Number of users: {}'.format(n_users))
print('Number of products: {}'.format(n_items))
products_by_freq.head()

Number of users: 2238
Number of products: 7113


,customer_key,product_name,count,customer_id,item_id
27811,45193aab-887a-4590-be6e-6558691e63b1,Mini Carnation (Spray) Hamada 50cm,196,0,0
27812,45193aab-887a-4590-be6e-6558691e63b1,Mini Carnation (Spray) Imagine 50cm,196,0,1
27813,45193aab-887a-4590-be6e-6558691e63b1,Mini Carnation (Spray) Pigeon 50cm,196,0,2
27814,45193aab-887a-4590-be6e-6558691e63b1,Mini Carnation (Spray) Pink Pigeon 50cm,196,0,3
27815,45193aab-887a-4590-be6e-6558691e63b1,Mini Carnation (Spray) Rony 50cm,196,0,4


El uso de este algoritmo llamado "Alternating Least Squares algorithm", se da para manejar matrices dispersas, en este caso, nuestro dataset no cuenta con valoraciones para todos los productos de manera directa, por lo tanto el uso de esta implementacion permite predicir un ranking o puntacion para aquellos productos que aun no han sido usados, seleccionados o analizados por un usuario.

Ahora procedemos a crear nuestra matriz dispersa que relaciona los cliente con los productos. En las cuales vamos asignar como valores las frecuencias de compras que tiene cada cliente por cada producto.


In [13]:
row = products_by_freq['customer_id'].values
col = products_by_freq['item_id'].values
data = np.ones(products_by_freq.shape[0])
coo_train = coo_matrix((data, (row, col)), shape=(len(unique_customers.tolist()), len(unique_items.tolist())))

Luego de obtener nuestra matriz de dispersion, vamos a inicializar y entrenar nuestro modelo, para eso vamos a utilizar la libreria implicit, la cual provee diferentea algorimos populares para sistemas de recomendacion para datasets con Retroalimentacion implicita, en este caso usaremos "AlternatingLeastSquares".

En este modelo, se tiene como objetivo conocer la interaccion entre un usuario y un elemento, se utilizan los valores de una matriz dispersa que representan si un usuario a tenido interaccion con un elemento o no la ha tenido. Los valores analizados en esas matriz son conocidos como confianza, en este caso la confianza sera la frecuencia de compras de un usuario basado en un producto.

Para ello definimos tres hiperparametros: 

1. factors: Los factores o factores latentos son propiedades ocultas que estan implicitas en nuestro dataset que infieren en atributos "ocultos" que no son visibles pero pueden influir en porque gusta mas un producto o no. En este caso lo inicializamos con valor en 20. Se define este valor para evitar sobre ajuste debido que tendria en cuenta muchos atributos a la hora de analizar los productos y un mayor costo de computo.
2. regularization: Este hiperparametro permite mantener un equilibrio entre el sobreajuste y el subajuste del modelo, ya que si el valor es muy alto, hace el modelo mas simple pero puede que no se tenga en cuenta muchas relaciones y si es muy bajo puede haber riesgo de sobre ajuste y no haga una buena generalizacion entre los productos
3. random_state: Permite colocar una semilla aleatoria para siempre conseguir reproducir los experimentos en el mismo orden y mantener la consistencia en ellos

In [14]:
# initialize a model
model = implicit.als.AlternatingLeastSquares(factors=50, regularization=1, random_state=123)
model.fit(coo_train)

  0%|          | 0/15 [00:00<?, ?it/s]

Luego de obtener las matrices dispersas, con las relaciones entre los productos y los clientes que los valores implicitos asignados usando la frecuencia de compra, vamos a entrenar y validar nuestro modelo para obtener los mejores hiperparametros posibles para nuestro conjunto de datos y para ello se organizara de la siguiente manera:

1. Se creó una función llamada `sparse_customer_item` donde dado el conjunto de datos preparado con los IDs para los productos y clientes, crea la matriz de dispersion.
2. Para el entrenamiento del modelo usara la tenica de validacion cruzada o "Cross validation", en la cual separamos el conjunto de datos en dos, una parte para entrenar el modelo y otra es reservada para la validacion de nuestro modelo. Para ello utilizaremos la libreria `sklearn`, la cual es una libreria nos ofrece un funcion llamada `train_test_split`, la cual dado un conjunto de datos nos permite separarlo asignando un porcentaje para entrenamiento y otro para pruebas.
3. Se creó una funcion que nos permite manejar los datos de validacion, entrenamiento y la matrix de dispersion para usar sus valores durante el proceso de validacion del modelo.

Finalmente, luego de crear las funciones, se procede a la validacion del modelo a traves de la optimizacion de los hiperparametros a traves de la tecnica de **Grid Search**. Esta técnica consiste en probar todas las combinaciones posibles de un conjunto predefinido de valores de hiperparámetros y seleccionar la combinación que produce los mejores resultados según alguna métrica de evaluación, para este modelo se seleccionó la siguiente: **Mean Average Precision at k (MAP@k)**.

Es importante recalcar que se esta buscando los 10 productos mas relevantes para cada cliente. Es decir K = 10

In [15]:
def filter_sparse_matrix(df, min_user_interactions=5, min_item_interactions=5):
#     # Filtrar usuarios con pocas interacciones
    print(df.shape)
    user_counts = df['customer_id'].value_counts()
    df_filtered = df[df['customer_id'].isin(user_counts[user_counts >= min_user_interactions].index)]
    
    # Filtrar items con pocas interacciones
    item_counts = df_filtered['item_id'].value_counts()
    df_filtered = df_filtered[df_filtered['item_id'].isin(item_counts[item_counts >= min_item_interactions].index)]

    print(df_filtered.shape)
    return df_filtered

def sparse_customer_item(df):
    row = df['customer_id'].values
    col = df['item_id'].values
    data = np.ones(df.shape[0])
    coo = coo_matrix((data, (row, col)), shape=(len(unique_customers.tolist()), len(unique_items.tolist())))
    return coo

def split_data(df):
    df_train, df_val = train_test_split(df, test_size=0.1)
    return df_train, df_val

def get_sparse_matrix(df, min_user_interactions=125, min_item_interactions=55):
    df_filtered = filter_sparse_matrix(df, min_user_interactions, min_item_interactions)
    df_train, df_val = split_data(df_filtered)
    coo_train = sparse_customer_item(df_train)
    coo_val = sparse_customer_item(df_val)

    csr_train = coo_train.tocsr()
    csr_val = coo_val.tocsr()
    
    return {'coo_train': coo_train,
            'csr_train': csr_train,
            'csr_val': csr_val}



def validate(matrix, factors=200, iterations=20, regularization=0.01, show_progress=True):

    coo_train, csr_train, csr_val = matrix['coo_train'], matrix['csr_train'], matrix['csr_val']
    
    model = implicit.als.AlternatingLeastSquares(factors=factors, 
                                                 iterations=iterations, 
                                                 regularization=regularization, 
                                                 random_state=42)
    model.fit(coo_train, show_progress=show_progress)
    
    map10 = mean_average_precision_at_k(model, csr_train, csr_val, K=10, show_progress=show_progress)

    return map10


matrix = get_sparse_matrix(products_by_freq)

best_map10 = 0

for factors in [20, 50, 70]:
    for iterations in [12,15,20]:
        for regularization in [0.01, 0.05, 0.1, 1]:
            map10 = validate(matrix, factors, iterations, regularization, show_progress=False)
            if map10 > best_map10:
                best_map10 = map10
                best_params = {'factors': factors, 'iterations': iterations, 'regularization': regularization}


print(f"Best MAP@10: {best_map10*100}")

(102199, 5)
(8065, 5)
Best MAP@10: 13.353409465418608


Luego de realizar la validacion de nuestro modelo obtuvimos que los mejores parametros son: 

In [16]:
best_params

{'factors': 20, 'iterations': 15, 'regularization': 1}

Luego de obtener los mejores parametros posibles para nuestro modelo, se procede entrenar el modelo con el conjunto de datos

In [17]:
del matrix

coo_train = sparse_customer_item(products_by_freq)
csr_train = coo_train.tocsr()

def train(coo_train, factors=100, iterations=15, regularization=0.01, show_progress=True):
    model = implicit.als.AlternatingLeastSquares(factors=factors, 
                                                 iterations=iterations, 
                                                 regularization=regularization, 
                                                 random_state=42)
    model.fit(coo_train, show_progress=show_progress)
    return model

model = train(coo_train, **best_params)

  0%|          | 0/15 [00:00<?, ?it/s]

Luego de crear nuestro modelo y haberlo entrenado con el conjunto de pruebas, procedemos a realizar recomendaciones, 
para ello vamos a usar la funcion `recommend` de nuestro modelo, la cual dada un usuario nos dará las mejores N recomendaciones para un usuario y nos retornara los IDs de los elementos y sus puntajes:

Crearemos nuestra funcion personalizada para obtener recomendaciones, donde dado un `customer_key` nos devolvera las 5 mejores recomendaciones para ese usuario: 

In [18]:
items_by_id = dict(list(enumerate(unique_items.tolist())))

def getProductRecomendations(customer):
    userId=customer_ids[customer]
    ids, scores = model.recommend(userId,  csr_train[userId], N=10)
    product_id = [items_by_id[x] for x in ids]
    result_table = pd.DataFrame({"product_id": product_id, "score": scores})
    return result_table

basedRecomendations = getProductRecomendations('45193aab-887a-4590-be6e-6558691e63b1')
basedRecomendations

,product_id,score
0,Delphinium Planet Blush Pink (white few blush ...,0.331579
1,Hebes Green 40cm,0.275470
2,Delphinium Planet Blush Pink (white few blush ...,0.272692
3,Anemone Bicolor White -Blush shade 35cm,0.269003
4,Anemone Bayola (blue-purple) 30cm,0.259817
5,Dianthus Star Snow Tessino 50cm,0.254283
6,Dianthus Raffine Petit Faye 50cm,0.249000
7,Dianthus Solomio Fiorino Leo 50cm,0.230028
8,Scabiosa Bon Bon Vainilla 50cm,0.216193
9,Ranunculus Elegance White 30cm,0.216111


# Evaluación

Para la evaluación de este modelo se utilizo la metrica mencionada anteriormente, **Mean Average Precision at k (MAP@k)**, la cual es una medida utilizada para evaluar la precisión de un sistema de recomendación o un modelo de recuperación de información, tomando en cuenta la posición de los elementos relevantes en las recomendaciones.

Es importante tener en cuenta que el algoritmo utilizado fue para un conjunto de datos con valoraciones implicitas,
donde las interacciones o parcipaciones del cliente con los diferentes productos de la plataforma no están explícitamente valoradas, dadas por una calificacion directa sobre un producto, este valor de MAP@k refleja la capacidad nuestro sistema de recomendacion para priorizar los elementos que los usuarios están más propensos a encontrar relevantes según su comportamiento.

El valor obtenido para MAP@k fue de: 

In [19]:
print(best_map10)
perc = round(best_map10 * 100)
print(f' Precision: {perc}%')

0.1335340946541861
 Precision: 13%


Lo cual quiere decir que aproximadamente el 6.6% de los primeros K(10) productos recomendados son relevantes para el usuario, es importante recordar que solo se estan teniendo en cuenta valoraciones implicitas y que a futuro se pueden tener en cuenta otras valoraciones para mejorar el modelo.

# Limitantes y propuesta para solucionarlo

Este modelo solo funciona para usuarios dentro del conjunto de datos provisto, cualquier usuario nuevo que llegue y no cuente con historial de compras, no podra obtener una sugerencia de los productos, esto limita el sistema hasta este punto debido que se quiere hacer todas las posibles sugerencias a todos los posibles productos. Para ello se ha decido complementar este modelo con el uso de la metrica de la similitud del coseno, para asi evaluar las caracteristicas del usuario nuevo con los existenes, encontrar el mas parecido a el y basado en el usuario encontrado, se daran recomendaciones.

Para ello procedemos a utilizar el conjunto de datos inicial con los usuarios y productos.

Primeramente, procedemos agrupar en un nuevo dataset los usuarios que compraron en todo el año 2023. Esto se hará para tener un registro unico de cada usuario y así hacer un analisis de sus caracteristicas y realizar una comparación mas precisa entre ellos.

In [20]:
copy_selected_group = copy_selected.drop(['product_group', 'variety', 'variety_key', 'length'], axis='columns')
columns = ['state', 'tier', 'business', 'events_per_year', 'spend_per_week', 'stores_quantity', 'employees_quantity', 'margin']
aggs = {key: ['first'] for key in columns}
copy_selected_group = copy_selected_group.groupby(by=["customer_key"]).agg(aggs).droplevel(0,axis=1).reset_index().set_index('customer_key') 
copy_selected_group.columns = columns
print(copy_selected_group.shape) 
copy_selected_group.head()

(2238, 8)


,state,tier,business,events_per_year,spend_per_week,stores_quantity,employees_quantity,margin
customer_key,,,,,,,,
000d1beb-2fa6-44cf-a68a-54c369090875,Mississippi,tier_2,Florist shop,10-24,$100-$499,1.0,4,36
007a32b6-b1a7-4c9d-9cb3-937ecc1f603d,South Carolina,tier_3,Other,< 10,< $100,1.0,3,42
009a8305-205b-4337-a34a-8d423a8c7704,Arkansas,tier_1,"Event planner (only), not a floral designer or...",> 60,Unknown,1.0,5,31
00a5424e-601d-486b-81b3-0f7db37b3a32,Wisconsin,tier_2,Florist shop,10-24,Unknown,1.0,1,36
01340b6f-7b0a-4e99-b088-efd37b549342,Virginia,tier_3,Wedding & event floral designer only (not a fl...,10-24,Unknown,0.0,2,42


# Analisis

### Inserción del nuevo customer

Se iniciará el proceso para hayar la recomendacion de productos para un usuario nuevo basado en las compras de otro usuario cuyas caracteristicas sean similares a este. Esto se hará a traves de una matriz de similitud de coseno.

La matriz de similitud de coseno es una herramienta que se utiliza para medir la similitud entre usuarios en función de sus características. Esta matriz se construye calculando el coseno del ángulo entre los vectores que representan a cada usuario en un espacio multidimensional.

Para este analisis se usará un usuario que se creo previamente en la plataforma y no cuenta con historial de compra, se extrajo las caracteristicas mencionadas al inicio del cuaderno y se insertó como un elemento mas del dataframe que incluye la informacion de los usuarios que han realizado compras. Asi mismo se realizan las verificacion y tratamientos de datos faltantes para asegurar que este validado al igual que el conjunto inicial.

In [21]:
customer_key = "c7792ee4-7303-4919-9521-1f221d5bdd96"

new_customer = {
    "state": "Texas",
    "tier": "Tier_2",
    "business": "Wedding & event floral designer only (not a florist)",
    "events_per_year": "25 - 59",
    "spend_per_week": "Unknown",
    "stores_quantity": "1",
    "employees_quantity": "8",
    "margin": 31
}

new_df = pd.DataFrame(new_customer, index=[customer_key])

copy_new_df = new_df.copy()

copy_new_df.loc[copy_new_df['spend_per_week'] == "",['spend_per_week']] =  'Unknown'
copy_new_df.loc[copy_new_df['spend_per_week'] != copy_new_df['spend_per_week'],['spend_per_week']] =  'Unknown'
copy_new_df.loc[copy_new_df['events_per_year'] == "",['events_per_year']] =  'Unknown'
copy_new_df.loc[copy_new_df['stores_quantity'] == "",['stores_quantity']] =  0.0
copy_new_df.loc[copy_new_df['employees_quantity'] == "",['employees_quantity']] =  0.0
copy_new_df.loc[:,['stores_quantity']] = copy_new_df['stores_quantity'].fillna(0).astype(float)
copy_new_df.loc[:,['employees_quantity']] = copy_new_df['employees_quantity'].fillna(0).astype(float)

copy_selected_group = pd.concat([copy_selected_group, copy_new_df])
copy_selected_group

,state,tier,business,events_per_year,spend_per_week,stores_quantity,employees_quantity,margin
000d1beb-2fa6-44cf-a68a-54c369090875,Mississippi,tier_2,Florist shop,10-24,$100-$499,1.0,4.0,36
007a32b6-b1a7-4c9d-9cb3-937ecc1f603d,South Carolina,tier_3,Other,< 10,< $100,1.0,3.0,42
009a8305-205b-4337-a34a-8d423a8c7704,Arkansas,tier_1,"Event planner (only), not a floral designer or...",> 60,Unknown,1.0,5.0,31
00a5424e-601d-486b-81b3-0f7db37b3a32,Wisconsin,tier_2,Florist shop,10-24,Unknown,1.0,1.0,36
01340b6f-7b0a-4e99-b088-efd37b549342,Virginia,tier_3,Wedding & event floral designer only (not a fl...,10-24,Unknown,0.0,2.0,42
...,...,...,...,...,...,...,...,...
ffb513af-c969-4616-97f3-428de2c37032,Minnesota,tier_1,Florist shop,> 60,Unknown,1.0,6.0,36
fff1017d-becb-4b75-9bcf-ba957544ca6b,Tennessee,tier_2,Wedding & event floral designer only (not a fl...,10-24,Unknown,1.0,2.0,31
fff3ca77-b401-4329-a6c1-df4a2ed3462c,New York,tier_3,Wedding & event floral designer only (not a fl...,< 10,$100-$499,0.0,1.0,42
fff60109-7599-4cc1-8c42-7f44cc0bcdc8,Nevada,tier_3,Florist shop,10 - 24,Unknown,0.0,2.0,42


Luego de haber ingresado el nuevo usuario y verificar que ingresó a nuestro dataset de customers. Procederemos a convertir nuestras columnas categoricas en numericos para aplicar la similitud de coseno sobre nuestro dataset de vectores que vamos a obtener a continuacion con el uso de la funcion `get_dummies` de la libreria pandas. Está función nos provee un nuevo dataset donde cada column representa una categoría y un valor, donde 1 es equivalente a la presencia esa categoría y un 0 su ausencia.

In [22]:
data = pd.get_dummies(copy_selected_group, columns=['state', 'tier', 'business', 'events_per_year', 'spend_per_week'])
data

,stores_quantity,employees_quantity,margin,state_Alabama,state_Alaska,state_Arizona,state_Arkansas,state_California,state_Colorado,state_Connecticut,...,events_per_year_10-24,events_per_year_25 - 59,events_per_year_25-59,events_per_year_< 10,events_per_year_> 60,spend_per_week_$100-$499,spend_per_week_$500-$999,spend_per_week_< $100,spend_per_week_> $1000,spend_per_week_Unknown
000d1beb-2fa6-44cf-a68a-54c369090875,1.0,4.0,36,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
007a32b6-b1a7-4c9d-9cb3-937ecc1f603d,1.0,3.0,42,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
009a8305-205b-4337-a34a-8d423a8c7704,1.0,5.0,31,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,1
00a5424e-601d-486b-81b3-0f7db37b3a32,1.0,1.0,36,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
01340b6f-7b0a-4e99-b088-efd37b549342,0.0,2.0,42,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffb513af-c969-4616-97f3-428de2c37032,1.0,6.0,36,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
fff1017d-becb-4b75-9bcf-ba957544ca6b,1.0,2.0,31,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
fff3ca77-b401-4329-a6c1-df4a2ed3462c,0.0,1.0,42,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
fff60109-7599-4cc1-8c42-7f44cc0bcdc8,0.0,2.0,42,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


# Aplicación de la Similitud de Coseno

Al obtener el dataset deseado cuyas columnas son valores numericos, se aplicará la similutud de coseno. Teniendo en cuenta el coseno del angulo entre dos vectores, representando asi las preferencias o gustos entre dos usuarios dando como resultado la similitud entre ellos.

Para esta aplicación se usara la función `cosine_similarity` de Scikit-learn, esta calcula la similitud de coseno entre dos conjuntos de vectores. 

Para facilitar la compresión de los datos y obtener mejores resultados se colocaron como index y columnas los key de los customer y asi hacer un mejor filtrado de sus valores y obtener los resultados mas cercanos entre si.

In [23]:
cos_sim = pd.DataFrame(cosine_similarity(data), columns=data.index, index=data.index)
cos_sim.head(100)

,000d1beb-2fa6-44cf-a68a-54c369090875,007a32b6-b1a7-4c9d-9cb3-937ecc1f603d,009a8305-205b-4337-a34a-8d423a8c7704,00a5424e-601d-486b-81b3-0f7db37b3a32,01340b6f-7b0a-4e99-b088-efd37b549342,01b79024-b222-44d6-be1e-d59b39fca976,01db02a5-dd1f-414c-b986-3ba5b5becf2c,01f4aa00-2167-4b97-adad-88be3aa656c2,0234cb76-9bb1-48e5-9534-b088bc64047f,026fcab4-14ad-416f-893a-8f13c30e1bbe,...,ff27820b-613f-4d1b-9187-79111a561a9c,ff294c02-a03c-4377-b2d0-be2fb8f3aded,ff37f19c-3f89-4b10-bcbd-3e198db9ecd1,ff91b5eb-49a2-4f4e-92c4-b48761bfe953,ff97d14f-f888-40e5-b9fd-b0e08c2f55e5,ffb513af-c969-4616-97f3-428de2c37032,fff1017d-becb-4b75-9bcf-ba957544ca6b,fff3ca77-b401-4329-a6c1-df4a2ed3462c,fff60109-7599-4cc1-8c42-7f44cc0bcdc8,c7792ee4-7303-4919-9521-1f221d5bdd96
000d1beb-2fa6-44cf-a68a-54c369090875,1.000000,0.995919,0.994368,0.995056,0.994986,0.995942,0.993339,0.997876,0.995641,0.995819,...,0.995852,0.994239,0.996200,0.995110,0.996965,0.995510,0.996223,0.993210,0.994986,0.985673
007a32b6-b1a7-4c9d-9cb3-937ecc1f603d,0.995919,1.000000,0.992137,0.995724,0.997186,0.996193,0.995978,0.997477,0.997186,0.996610,...,0.995712,0.997186,0.997757,0.995216,0.995919,0.992337,0.995960,0.996904,0.997186,0.979837
009a8305-205b-4337-a34a-8d423a8c7704,0.994368,0.992137,1.000000,0.987761,0.990044,0.993428,0.987829,0.994698,0.990044,0.995265,...,0.990888,0.986831,0.994226,0.987829,0.996991,0.998192,0.991397,0.986355,0.990044,0.991781
00a5424e-601d-486b-81b3-0f7db37b3a32,0.995056,0.995724,0.987761,1.000000,0.997407,0.994560,0.997259,0.996393,0.998065,0.995414,...,0.997120,0.997971,0.994705,0.999040,0.994293,0.988349,0.997496,0.996277,0.997407,0.971487
01340b6f-7b0a-4e99-b088-efd37b549342,0.994986,0.997186,0.990044,0.997407,1.000000,0.995596,0.996136,0.996354,0.998872,0.996983,...,0.996633,0.997743,0.997757,0.996899,0.994986,0.990119,0.997644,0.998025,0.998308,0.976316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0c06aba0-7225-451c-85e8-98114c2b6a36,0.998076,0.995683,0.993935,0.996711,0.995846,0.996412,0.994864,0.998274,0.996606,0.996317,...,0.996921,0.994887,0.996281,0.996922,0.997195,0.994964,0.997435,0.992885,0.995846,0.983927
0c2581f1-30a8-42bf-a75d-56947d22094c,0.994986,0.997186,0.990044,0.997407,0.999436,0.995596,0.996899,0.996354,0.998872,0.996983,...,0.996633,0.997743,0.997757,0.996899,0.994986,0.990119,0.997644,0.998025,0.998308,0.976316
0c368a87-749b-4703-9844-db374b8b6573,0.993193,0.995637,0.987190,0.997476,0.997928,0.994940,0.996886,0.995501,0.997320,0.995841,...,0.996679,0.996667,0.995780,0.996886,0.993193,0.987142,0.996987,0.997557,0.997320,0.971527
0c3cb544-ff60-4f68-a4b2-da378335a39b,0.995431,0.997959,0.993935,0.994051,0.996606,0.996412,0.994864,0.997517,0.995846,0.996317,...,0.994866,0.994126,0.997796,0.993836,0.996313,0.994089,0.995381,0.994407,0.995846,0.983927


Al aplicar la similitud de coseno pudimos obtener nuestro data frame resultante con los valores de similitud. En los indices se ubica cada customer key y cada columna representa todos los usuarios (por keys) con los que se realizó la comparación y los valores por cada uno de ellos.

# Resultados

En primer lugar, luego de obtener nuestros resultados se tendrá en cuenta el usuario que se agregó al final del dataset, debido que este será el que obtendrá sus recomendaciones antes de realizar el paso final para completar su compra. Este usuario será referenciado por su `customer_key = c7792ee4-7303-4919-9521-1f221d5bdd96`

Luego, obtendremos de nuestra matriz de similitud todos los usuarios dados por el indice principal, el usuario nuevo. Al extraer la infomacion, ordenamos los valores de similitud de mayor a menor excluyendo el sujeto principal debido que tiene 100% de compatibilidad. Finalmente se conseguirá una matriz con los 5 valores mas altos y se tomará el primer valor como usuario mas compatible.

In [24]:
print(f"New customer key: {customer_key}")
sim_customers = cos_sim[customer_key].iloc[lambda x: x.index != customer_key].sort_values(ascending=False).head(5)
df_sim_customers = pd.DataFrame(sim_customers)

sim_customer = sim_customers.index[0]


print(f"\nMore similar customer: {sim_customer}")

df_sim_customers.head()

New customer key: c7792ee4-7303-4919-9521-1f221d5bdd96

More similar customer: 2f270530-43db-4e91-ad01-0eed173a68af


,c7792ee4-7303-4919-9521-1f221d5bdd96
2f270530-43db-4e91-ad01-0eed173a68af,0.998109
aa702eff-11c6-4360-9cea-cb86e3a3ed75,0.998060
d743c173-4b89-41f2-ba35-be058cc6f847,0.997405
d15ee675-dc12-48e7-914c-d80974e2f003,0.997405
b57d0f5e-2177-416a-a919-85cbfe8aa8d7,0.997405


A continuación, se hara una muestra de los usuarios resultantes para realizar una comparación entre sus caracteristicas y analizar la efectividad de nuestra similitud.

In [25]:
pd.DataFrame([copy_selected_group.loc[sim_customer], copy_selected_group.loc[customer_key]])

,state,tier,business,events_per_year,spend_per_week,stores_quantity,employees_quantity,margin
2f270530-43db-4e91-ad01-0eed173a68af,Washington,tier_2,Wedding & event floral designer only (not a fl...,25 - 59,Unknown,1.0,10.0,36
c7792ee4-7303-4919-9521-1f221d5bdd96,Texas,Tier_2,Wedding & event floral designer only (not a fl...,25 - 59,Unknown,1.0,8.0,31


Al encontrar el usuario mas similar al nuevo, procedemos a utilizar nuestro modelo de recomendacion para obtener los productos:

In [26]:
newUserRecomendations = getProductRecomendations(sim_customer)
newUserRecomendations

,product_id,score
0,Queen Annes Lace White QAL 70cm,0.026580
1,Roses Playa Blanca (Sometimes have subtle blu...,0.024967
2,Ruscus Israeli 60cm,0.022415
3,Ranunculus Chocolato Natural 35cm,0.022306
4,Ranunculus White 30cm,0.022022
5,Asparagus Plumosus 40cm,0.020367
6,Amaranthus Green hanging 60cm,0.020286
7,Lisianthus Double Pink 50cm,0.019769
8,Hydrangea White Premium 18cm head size 60cm,0.019393
9,Amaranthus Hot Biscuits 70cm,0.019315


# Visualización de los resultados

Para visualizar los resultados con un mayor detalle, primero crearemos una funcion que nos permitirá visualizar las imagenes de los productos.

In [27]:
def path_to_image_html(path):
    '''
     This function essentially convert the image url to 
     '<img src="'+ path + '"/>' format. And one can put any
     formatting adjustments to control the height, aspect ratio, size etc.
     within as in the below example. 
    '''

    return '<img src="'+ path + '" style=max-height:124px;"/>'

Ahora vamos a obtener el conjunto de datos para la previsualizacion de las recomendaciones que vera el usuario en la plataforma:

Primeramente obtenemos todo el conjunto de datos y creamos la columna que servira como ID para comparar el conjunto de datos con nuestras recomendaciones: 

In [28]:
copy_products = copy_selected.copy()
copy_products['length'].fillna(0, inplace=True)
copy_products['product_name'] = copy_products.apply(lambda x: str(x.product_group) + " " + str(x.variety) + " " + str(int(float(x.length)))+ "cm", axis = 1)

Luego creamos una funcion que nos provee un dataframe que solo contiene el nombre de los productos recomendado y su imagen, recibe como parametro las recomendaciones:

In [31]:
def getRecommenderListPreview(recomendations):
    copy_rec = recomendations.copy()
    copy_rec.rename(columns={ 'product_id': 'product_name'}, inplace=True)

    df_merge = pd.merge(copy_rec, copy_products[['product_name', 'variety_key']], on='product_name', how='left')
    df_merge = df_merge.drop_duplicates()
    df_merge['image'] = df_merge.apply(lambda x:  "https://sbxcloud.com/www/ibuyflowers/varieties/variety_" + x.variety_key + ".jpg", axis = 1)
    df_merge['image_preview'] = df_merge['image'].apply(lambda f: path_to_image_html(f))
    return df_merge[['product_name', 'image_preview']]

# Example
getRecommenderListPreview(basedRecomendations)

,product_name,image_preview
0,Delphinium Planet Blush Pink (white few blush ...,"<img src=""https://sbxcloud.com/www/ibuyflowers..."
321,Hebes Green 40cm,"<img src=""https://sbxcloud.com/www/ibuyflowers..."
522,Delphinium Planet Blush Pink (white few blush ...,"<img src=""https://sbxcloud.com/www/ibuyflowers..."
717,Anemone Bicolor White -Blush shade 35cm,"<img src=""https://sbxcloud.com/www/ibuyflowers..."
908,Anemone Bayola (blue-purple) 30cm,"<img src=""https://sbxcloud.com/www/ibuyflowers..."
1064,Dianthus Star Snow Tessino 50cm,"<img src=""https://sbxcloud.com/www/ibuyflowers..."
1223,Dianthus Raffine Petit Faye 50cm,"<img src=""https://sbxcloud.com/www/ibuyflowers..."
1387,Dianthus Solomio Fiorino Leo 50cm,"<img src=""https://sbxcloud.com/www/ibuyflowers..."
1528,Scabiosa Bon Bon Vainilla 50cm,"<img src=""https://sbxcloud.com/www/ibuyflowers..."
1727,Ranunculus Elegance White 30cm,"<img src=""https://sbxcloud.com/www/ibuyflowers..."


Luego de obtener nuestros productos, se graficarán para mostrar el resultado del desarrollo propuesto para obtener recomendaciones para usuarios nuevos basados en sus caracteristicas al momento de realizar el registro en la plataforma de [iBuyFlowers](beta.ibuyflowers.com).

Teniendo en cuenta lo anterior, daremos una mirada a los productos obtenidos con sus imagenes y finalmente, graficamos los productos que vamos a recomendar (top 10):

Pero vamos a visualizar los obtenidos al principio del desarrollo con un usuario previamente registrado:

In [32]:
from IPython.display import HTML

oldCustomer = getRecommenderListPreview(basedRecomendations)
HTML(oldCustomer.to_html(escape=False ,formatters=dict(column_name_with_image_links=path_to_image_html)))

,product_name,image_preview
0,Delphinium Planet Blush Pink (white few blush touches) 50cm,
321,Hebes Green 40cm,
522,Delphinium Planet Blush Pink (white few blush touches) 60cm,
717,Anemone Bicolor White -Blush shade 35cm,
908,Anemone Bayola (blue-purple) 30cm,
1064,Dianthus Star Snow Tessino 50cm,
1223,Dianthus Raffine Petit Faye 50cm,
1387,Dianthus Solomio Fiorino Leo 50cm,
1528,Scabiosa Bon Bon Vainilla 50cm,
1727,Ranunculus Elegance White 30cm,


Ahora vamos a revisar el usuario obtenido por la similitud del coseno:

In [33]:
newCustomer = getRecommenderListPreview(newUserRecomendations)
HTML(newCustomer.to_html(escape=False ,formatters=dict(column_name_with_image_links=path_to_image_html)))

,product_name,image_preview
0,Queen Annes Lace White QAL 70cm,
434,Roses Playa Blanca (Sometimes have subtle blush hints) 40cm,
1243,Ruscus Israeli 60cm,
1944,Ranunculus Chocolato Natural 35cm,
2088,Ranunculus White 30cm,
2425,Asparagus Plumosus 40cm,
2613,Amaranthus Green hanging 60cm,
2758,Lisianthus Double Pink 50cm,
2898,Hydrangea White Premium 18cm head size 60cm,
3415,Amaranthus Hot Biscuits 70cm,
